<a href="https://colab.research.google.com/github/pedrohahn/apachespark-deltalake-iceberg/blob/main/wine/Trabalho%20de%20Pesquisa%20-%20Dataset%20Wine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalando as dependências
!pip install delta-spark
!pip install py4j
!pip install apache-iceberg

# Importando as bibliotecas necessárias
from pyspark.sql import SparkSession

# Iniciando uma sessão Spark
spark = SparkSession.builder \
    .appName("wine_data") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

# Lendo o dataset
wine_df = spark.read.option("header", True).csv("winemag-data-130k-v2.csv")

# Mostrando as primeiras linhas do DataFrame
wine_df.show(5)

# Escrevendo o DataFrame como uma tabela Delta
wine_df.write.format("delta").save("/tmp/delta_wine_data")

# Escrevendo o DataFrame como uma tabela Iceberg
wine_df.write.format("iceberg").save("/tmp/iceberg_wine_data")

DELTA LAKE

In [ ]:
# Lendo a tabela Delta
delta_wine_df = spark.read.format("delta").load("/tmp/delta_wine_data")

# Criando um novo DataFrame para inserção
new_wine_data = spark.read.option("header", True).csv("new_wine_data.csv")

# Realizando a operação de INSERT
new_wine_data.write.format("delta").mode("append").save("/tmp/delta_wine_data")

In [ ]:
# Atualizando dados na tabela Delta
from pyspark.sql.functions import col

delta_wine_df = delta_wine_df.withColumn("price", col("price") * 1.1)

delta_wine_df.write.format("delta").mode("overwrite").save("/tmp/delta_wine_data")

In [ ]:
# Deletando dados na tabela Delta
delta_wine_df = delta_wine_df.filter(col("country") != "Portugal")

delta_wine_df.write.format("delta").mode("overwrite").save("/tmp/delta_wine_data")

ICEBERG

In [ ]:
# Lendo a tabela Iceberg
iceberg_wine_df = spark.read.format("iceberg").load("/tmp/iceberg_wine_data")

# Criando um novo DataFrame para inserção
new_wine_data = spark.read.option("header", True).csv("new_wine_data.csv")

# Realizando a operação de INSERT
new_wine_data.write.format("iceberg").mode("append").save("/tmp/iceberg_wine_data")

In [ ]:
# Atualizando dados na tabela Iceberg
from pyspark.sql.functions import col

iceberg_wine_df = iceberg_wine_df.withColumn("price", col("price") * 1.1)

iceberg_wine_df.write.format("iceberg").mode("overwrite").save("/tmp/iceberg_wine_data")

In [ ]:
# Deletando dados na tabela Iceberg
iceberg_wine_df = iceberg_wine_df.filter(col("country") != "Portugal")

iceberg_wine_df.write.format("iceberg").mode("overwrite").save("/tmp/iceberg_wine_data")